In [1]:
import os
os.listdir()

['DEX-K3CT42143005277-234200-041725-SCHEDULED.log']

In [3]:
def parse_dex_log(file_path):
    with open(file_path, 'r') as f:
        content = f.read()

    # Split by lines
    lines = [line.strip() for line in content.split('\n') if line.strip()]
    
    decoded = {}
    for line in lines:
        parts = line.split('*')
        record_type = parts[0]

        if record_type not in decoded:
            decoded[record_type] = []

        decoded[record_type].append(parts[1:])

    return decoded


def print_decoded_dex(decoded):
    for record, entries in decoded.items():
        print(f"\n=== {record} ===")
        for entry in entries:
            print(entry)


# Example usage:
dex_file = "DEX-K3CT42143005277-234200-041725-SCHEDULED.log"  # Replace with your actual .log file path
decoded = parse_dex_log(dex_file)
print_decoded_dex(decoded)



=== DXS ===
['WTN4213952', 'VA', 'V0/6', '1']

=== ST ===
['001', '0001']

=== ID1 ===
['WTN11240170009', 'GVC1        ', '9421', '', '', '']

=== ID4 ===
['2', '001', '5']

=== CB1 ===
['11240170009', 'GVC1        ', '9421']

=== VA1 ===
['1725', '12', '1725', '12', '0', '0', '0', '0']

=== VA2 ===
['900', '11', '900', '11']

=== VA3 ===
['1050', '14', '1050', '14']

=== CA1 ===
['MAK237S11622   ', 'MCM5        ', '0020', '', '']

=== BA1 ===
['CPI5030GA01710 ', 'CV3         ', '1200', '', '']

=== DA1 ===
['CAS000000000001', '000000000001', '0001', '', '']

=== CA2 ===
['775', '6', '775', '6']

=== CA3 ===
['2280', '0', '1480', '8', '2280', '0', '1480', '8', '800', '800']

=== CA4 ===
['250', '25', '250', '25']

=== CA8 ===
['0', '0']

=== CA9 ===
['0', '0', '0', '0']

=== CA10 ===
['1280', '1280']

=== CA15 ===
['1255', 'TUBE1', '19', '16', '36', '1', '0', '0', '0', '0']
['1255', 'TUBE2', '0', '0', '0', '0', '0', '0', '0', '0']

=== CA16 ===
['0', '0']

=== CA17 ===
['0', '5', '19'

In [1]:
import csv
import pandas as pd

def parse_dex_log_to_dataframe(file_path):
    with open(file_path, 'r') as f:
        content = f.read()

    lines = [line.strip() for line in content.split('\n') if line.strip()]
    
    records = []
    for line in lines:
        parts = line.split('*')
        record_type = parts[0]
        fields = parts[1:]
        records.append((record_type, fields))
    
    return records

def extract_product_sales(records):
    product_data = []
    current_product = {}
    for record_type, fields in records:
        if record_type == "PA1":
            current_product = {
                "Slot": fields[0],
                "Price": fields[1]
            }
        elif record_type == "PA2" and current_product:
            current_product.update({
                "Vends_Cash": int(fields[0]) if fields[0].isdigit() else 0,
                "Cash_Sales_Cents": int(fields[1]) if fields[1].isdigit() else 0,
                "Vends_Cashless": int(fields[2]) if fields[2].isdigit() else 0,
                "Cashless_Sales_Cents": int(fields[3]) if fields[3].isdigit() else 0,
            })
        elif record_type == "PA5" and current_product:
            current_product["Last_Sale_Date"] = fields[0] if len(fields) > 0 else ""
            product_data.append(current_product)
            current_product = {}

    return pd.DataFrame(product_data)

# Load and process sample file
file_path = 'DEX-K3CT42143005277-234200-041725-SCHEDULED.log'
records = parse_dex_log_to_dataframe(file_path)
product_df = extract_product_sales(records)

# Generate summary stats
summary = {
    "Total Vends (Cash)": product_df["Vends_Cash"].sum(),
    "Total Vends (Cashless)": product_df["Vends_Cashless"].sum(),
    "Total Sales (Cash) $": product_df["Cash_Sales_Cents"].sum() / 100.0,
    "Total Sales (Cashless) $": product_df["Cashless_Sales_Cents"].sum() / 100.0,
    "Total Sales $": (product_df["Cash_Sales_Cents"].sum() + product_df["Cashless_Sales_Cents"].sum()) / 100.0
}

# Save to CSV
csv_output_path = 'dex_sales_report.csv'
product_df.to_csv(csv_output_path, index=False)



In [2]:
product_df

,Slot,Price,Vends_Cash,Cash_Sales_Cents,Vends_Cashless,Cashless_Sales_Cents,Last_Sale_Date
0,010,150,8,975,8,975,20020316
1,011,150,2,300,2,300,
2,012,150,0,0,0,0,
3,013,250,1,250,1,250,20020313
4,014,100,0,0,0,0,
5,015,200,1,200,1,200,20020312
6,016,100,0,0,0,0,
7,017,100,0,0,0,0,
8,018,150,0,0,0,0,
9,019,250,0,0,0,0,


In [3]:
summary

{'Total Vends (Cash)': 12,
 'Total Vends (Cashless)': 12,
 'Total Sales (Cash) $': 17.25,
 'Total Sales (Cashless) $': 17.25,
 'Total Sales $': 34.5}